In [ ]:
from dbnomics_client import fetch_dataframe, fetch_dataframe_from_url

Use the URL copy-pasted from DB.nomics website:

In [ ]:
df = fetch_dataframe_from_url("https://api.next.nomics.world/Eurostat/met_road_gonu?dimension=nst07%3AGT15")
df

Specify the `provider_code` and the `dataset_code` manually:

In [ ]:
df = fetch_dataframe(
    provider_code='Eurostat',
    dataset_code='met_road_gonu',
#     limit=30,
)

Find the number of series available in the `DataFrame`:

In [ ]:
len(set(df.code))